## Step 1: Quantize the model

In [8]:
!python -m awq.entry --model_path ~/.cache/huggingface/transformers/openchat_3.5  \
    --w_bit 4 --q_group_size 128 \
    --run_awq --dump_awq awq_cache/openchat_3.5-w4-g128.pt

Quantization config: {'zero_point': True, 'q_group_size': 128}
* Building model /PHShome/bg615/.cache/huggingface/transformers/openchat_3.5
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards: 100%|██████████████████| 2/2 [00:17<00:00,  8.84s/it]
Repo card metadata block was not found. Setting CardData to empty.
 * Split into 65 blocks
Running AWQ...: 100%|███████████████████████████| 32/32 [17:32<00:00, 32.91s/it]
AWQ results saved at awq_cache/openchat_3.5-w4-g128.pt


In [9]:
!python -m awq.entry --model_path ~/.cache/huggingface/transformers/openchat_3.5 \
    --w_bit 4 --q_group_size 128 \
    --load_awq awq_cache/openchat_3.5-w4-g128.pt \
    --q_backend real --dump_quant quant_cache/openchat_3.5-w4-g128.pt

Quantization config: {'zero_point': True, 'q_group_size': 128}
* Building model /PHShome/bg615/.cache/huggingface/transformers/openchat_3.5
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards: 100%|██████████████████| 2/2 [00:20<00:00, 10.02s/it]
Loading pre-computed AWQ results from awq_cache/openchat_3.5-w4-g128.pt
real weight quantization...: 100%|██████████████| 32/32 [04:09<00:00,  7.80s/it]
Saving the quantized model at quant_cache/openchat_3.5-w4-g128.pt...


## Step 2: Load and compare the quantized and unquantized model

In [1]:
from lm_eval import evaluator, tasks
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoConfig
import torch
import argparse
import os
import json
from accelerate import init_empty_weights, infer_auto_device_map, dispatch_model, load_checkpoint_in_model
from accelerate.utils.modeling import get_balanced_memory
from awq.utils.parallel import auto_parallel
from awq.quantize.pre_quant import run_awq, apply_awq
from awq.quantize.quantizer import pseudo_quantize_model_weight, real_quantize_model_weight
from awq.utils.lm_eval_adaptor import LMEvalAdaptor
from awq.utils.utils import simple_dispatch_model
import time
import gc

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [3]:
q_config = {
    "zero_point": True,  # by default True
    "q_group_size": 128,  # whether to use group quantization
}
max_memory = []

In [4]:
def build_model_and_enc(model_path, quantized_file_path, load_quant = True, w_bit = 4):
    if not os.path.exists(model_path):  # look into ssd
        raise FileNotFoundError(f"{model_path} not found!")
    print(f"* Building model {model_path}")

    # all hf model
    config = AutoConfig.from_pretrained(model_path, trust_remote_code=True)
    print(f"Config = {config}")
    if "mpt" in config.__class__.__name__.lower():
        enc = AutoTokenizer.from_pretrained(config.tokenizer_name, trust_remote_code=True)
    else:
        enc = AutoTokenizer.from_pretrained(model_path, use_fast=False, trust_remote_code=True)

    if load_quant:  # directly load quantized weights
        print("Loading pre-computed quantized weights...")
        with init_empty_weights():
            model = AutoModelForCausalLM.from_config(config=config,
                                                     torch_dtype=torch.float16, trust_remote_code=True)
        model.config.pretraining_tp = 1
        real_quantize_model_weight(
            model, w_bit=w_bit, q_config=q_config, init_only=True)
        
        model.tie_weights()
        
        # Infer device map
        kwargs = {"max_memory": max_memory} if len(max_memory) else {}
        device_map = infer_auto_device_map(
            model,
            no_split_module_classes=[
                "OPTDecoderLayer", "LlamaDecoderLayer", "BloomBlock", "MPTBlock", "DecoderLayer"],
            **kwargs
        )
        # Load checkpoint in the model
        load_checkpoint_in_model(
            model,
            checkpoint= quantized_file_path,
            device_map=device_map,
            offload_state_dict=True,
        )
        # Dispatch model
        model = simple_dispatch_model(model, device_map=device_map)

        model.eval()
    else:  # fp16 to quantized
        args.run_awq &= not args.load_awq  # if load_awq, no need to run awq
        # Init model on CPU:
        kwargs = {"torch_dtype": torch.float16, "low_cpu_mem_usage": True}
        model = AutoModelForCausalLM.from_pretrained(
            model_path, config=config, trust_remote_code=True, **kwargs)

        model.eval()

        if args.run_awq:
            assert args.dump_awq, "Please save the awq results with --dump_awq"
                        
            awq_results = run_awq(
                model, enc,
                w_bit=args.w_bit, q_config=q_config,
                n_samples=128, seqlen=512,
            )
            if args.dump_awq:
                dirpath = os.path.dirname(args.dump_awq)
                os.makedirs(dirpath, exist_ok=True)
                
                torch.save(awq_results, args.dump_awq)
                print("AWQ results saved at", args.dump_awq)
                
            exit(0)
                
        if args.load_awq:
            print("Loading pre-computed AWQ results from", args.load_awq)
            awq_results = torch.load(args.load_awq, map_location="cpu")
            apply_awq(model, awq_results)

        # weight quantization
        if args.w_bit is not None:
            if args.q_backend == "fake":
                assert args.dump_quant is None, \
                    "Need to use real quantization to dump quantized weights"
                pseudo_quantize_model_weight(
                    model, w_bit=args.w_bit, q_config=q_config
                )
            elif args.q_backend == "real":  # real quantization
                real_quantize_model_weight(
                    model, w_bit=args.w_bit, q_config=q_config
                )
                if args.dump_quant:
                    dirpath = os.path.dirname(args.dump_quant)
                    os.makedirs(dirpath, exist_ok=True)
                    
                    print(
                        f"Saving the quantized model at {args.dump_quant}...")
                    torch.save(model.cpu().state_dict(), args.dump_quant)
                    exit(0)
            else:
                raise NotImplementedError
            
        # Move the model to GPU (as much as possible) for LM evaluation
        kwargs = {"max_memory": get_balanced_memory(model, max_memory if len(max_memory) > 0 else None)}
        device_map = infer_auto_device_map(
            model,
            # TODO: can we remove this?
            no_split_module_classes=[
                "OPTDecoderLayer", "LlamaDecoderLayer", "BloomBlock", "MPTBlock", "DecoderLayer"],
            **kwargs
        )
        model = dispatch_model(model, device_map=device_map)

    return model, enc

In [5]:
model_path = "../.cache/huggingface/transformers/openchat_3.5"
quantized_file_path = "quant_cache/openchat_3.5-w4-g128.pt"
model, tokenizer = build_model_and_enc(model_path, quantized_file_path = quantized_file_path)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


* Building model ../.cache/huggingface/transformers/openchat_3.5
Config = MistralConfig {
  "_name_or_path": "../.cache/huggingface/transformers/openchat_3.5",
  "architectures": [
    "MistralForCausalLM"
  ],
  "bos_token_id": 1,
  "eos_token_id": 32000,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_position_embeddings": 8192,
  "model_type": "mistral",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 8,
  "rms_norm_eps": 1e-05,
  "rope_theta": 10000.0,
  "sliding_window": 4096,
  "tie_word_embeddings": false,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.35.0",
  "use_cache": true,
  "vocab_size": 32002
}

Loading pre-computed quantized weights...


real weight quantization...(init only): 100%|██████████████████████████████████████████████████████████████████████████████████████████████| 32/32 [00:00<00:00, 982.29it/s]


In [6]:
device = torch.device("cuda")
model.to(device)

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32002, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): WQLinear(in_features=4096, out_features=4096, bias=False, w_bit=4, group_size=128)
          (k_proj): WQLinear(in_features=4096, out_features=1024, bias=False, w_bit=4, group_size=128)
          (v_proj): WQLinear(in_features=4096, out_features=1024, bias=False, w_bit=4, group_size=128)
          (o_proj): WQLinear(in_features=4096, out_features=4096, bias=False, w_bit=4, group_size=128)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): WQLinear(in_features=4096, out_features=14336, bias=False, w_bit=4, group_size=128)
          (up_proj): WQLinear(in_features=4096, out_features=14336, bias=False, w_bit=4, group_size=128)
          (down_proj): WQLinear(in_features=14336, out_features=4096, bias=False, w_bit=4, group_

## Test the quantized model

In [7]:
def LLM_text(model, tokenizer, prompt):
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    generate_ids = model.generate(inputs.input_ids, max_length=512)
    print(generate_ids)
    question_and_response = tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
    response = question_and_response.split("ASSISTANT: ")[-1]
    return response

In [8]:
prompt = "What is the difference between Harvard and MIT?"
quant_start_time = time.time()
response = LLM_text(model, tokenizer, prompt)
print(response)
quant_end_time = time.time()
quant_run_time = quant_end_time - quant_start_time
print(f"[INFO]: The quantized model finishes running after {quant_run_time} seconds.")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:32000 for open-end generation.


tensor([[    1,  1824,   349,   272,  5133,  1444, 17553,   304, 17156, 28804,
            13,    13, 23653, 14743,  2900,   304,   272, 16255,  8624,   302,
         12511,   325, 24791, 28731,   460,   989,   302,   272,  1080, 15374,
         25390, 21356,   297,   272,  2969,  3543, 28723,  4023,  1560,  7373,
           460,  6416, 15390,   354,   652, 11860,  7034, 28725,   736,   460,
          2856,  1945, 11090,  1444,   272,   989, 28723,    13,    13, 28740,
         28723, 14716, 28747, 17553,  2900,   349,  5651,   297, 14751, 28725,
         16255, 28725,  1312, 17156,   349,   835,  5651,   297, 14751, 28725,
         16255, 28723,  7829,  7373,   460,   744,   302,   272, 14751, 28733,
         28760,  8410,  1424, 22159,  2698, 28725,   690,   349,  2651,   354,
           871,  6708,  3340, 28725,  5679, 28725,   304, 11860, 12638, 28723,
            13,    13, 28750, 28723, 13909, 28747, 17553,  2900,   659,   264,
          6084,  5716,  2187,   821, 17156, 28725,  

In [9]:
# Unload the model from GPU and RAM
del model          # Delete the model
gc.collect()       # Collect garbage
if device == 'cuda':
    torch.cuda.empty_cache()  # Clear CUDA cache

## This is what you will do if you want to load the original, unquantized model on GPU

In [10]:
tokenizer = AutoTokenizer.from_pretrained("/PHShome/bg615/.cache/huggingface/transformers/openchat_3.5")
model = AutoModelForCausalLM.from_pretrained("/PHShome/bg615/.cache/huggingface/transformers/openchat_3.5").half()
device = torch.device("cuda")
model.to(device)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32002, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): MistralRMSNorm()
        (post_attention_layernorm): MistralRMSNorm()
      )
    )
    (norm): MistralRM

## Test the unquantized model on GPU

In [11]:
def LLM_text(model, tokenizer, prompt):
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    generate_ids = model.generate(inputs.input_ids, max_length=512)
    print(generate_ids)
    question_and_response = tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
    response = question_and_response.split("ASSISTANT: ")[-1]
    return response

In [12]:
prompt = "What is the difference between Harvard and MIT?"
original_start_time = time.time()
response = LLM_text(model, tokenizer, prompt)
print(response)
original_end_time = time.time()
original_run_time = original_end_time - original_start_time
print(f"[INFO]: The original model finishes running after {original_run_time} seconds.")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:32000 for open-end generation.


tensor([[    1,  1824,   349,   272,  5133,  1444, 17553,   304, 17156, 28804,
            13,    13, 23653, 14743,  2900,   304,   272, 16255,  8624,   302,
         12511,   325, 24791, 28731,   460,   989,   302,   272,  1080, 15374,
         25390, 21356,   297,   272,  2969,  3543, 28723,  4023,   590,   460,
          1560,  6416, 15390, 12638, 28725,   736,   460,   741,  1945, 11090,
          1444,   272,   989, 28723,    13,    13, 23653, 14743,  2900,   349,
           264,  1597,   315, 12246,  6165,  3332, 10539,  5651,   297, 14751,
         28725, 16255, 28723,   661,   403, 11573,   297, 28705, 28740, 28784,
         28770, 28784,   304,   349,   272, 17690, 16854,   302,  4337,  5362,
           297,   272,  2969,  3543, 28723, 17553,   349,  2651,   354,   871,
          2967, 14987, 13480,  2007, 28725,   390,  1162,   390,   871,  5024,
          7373,   297,  2309, 28725,  1955, 28725,   304, 12502, 28723,   415,
         10539,  5751,   264,  5335,  2819,   302,  

In [13]:
# Unload the model from GPU and RAM
del model          # Delete the model
gc.collect()       # Collect garbage
if device == 'cuda':
    torch.cuda.empty_cache()  # Clear CUDA cache

## This is what you will do if you want to load the original, unquantized model on CPU

In [14]:
tokenizer = AutoTokenizer.from_pretrained("/PHShome/bg615/.cache/huggingface/transformers/openchat_3.5")
model = AutoModelForCausalLM.from_pretrained("/PHShome/bg615/.cache/huggingface/transformers/openchat_3.5")
device = torch.device("cpu")
model.to(device)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32002, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): MistralRMSNorm()
        (post_attention_layernorm): MistralRMSNorm()
      )
    )
    (norm): MistralRM

## Test the unquantized model on CPU

In [15]:
def LLM_text(model, tokenizer, prompt):
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    generate_ids = model.generate(inputs.input_ids, max_length=512)
    print(generate_ids)
    question_and_response = tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
    response = question_and_response.split("ASSISTANT: ")[-1]
    return response

In [16]:
prompt = "What is the difference between Harvard and MIT?"
original_cpu_start_time = time.time()
response = LLM_text(model, tokenizer, prompt)
print(response)
original_cpu_end_time = time.time()
original_cpu_run_time = original_cpu_end_time - original_cpu_start_time
print(f"[INFO]: The original CPU model finishes running after {original_cpu_run_time} seconds.")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:32000 for open-end generation.


tensor([[    1,  1824,   349,   272,  5133,  1444, 17553,   304, 17156, 28804,
            13,    13, 23653, 14743,  2900,   304,   272, 16255,  8624,   302,
         12511,   325, 24791, 28731,   460,   989,   302,   272,  1080, 15374,
         25390, 21356,   297,   272,  2969,  3543, 28723,  4023,   590,   460,
          1560,  6416, 15390, 12638, 28725,   736,   460,   741,  1945, 11090,
          1444,   272,   989, 28723,    13,    13, 23653, 14743,  2900,   349,
           264,  1597,   315, 12246,  6165,  3332, 10539,  5651,   297, 14751,
         28725, 16255, 28723,   661,   403, 11573,   297, 28705, 28740, 28784,
         28770, 28784,   304,   349,   272, 17690, 16854,   302,  4337,  5362,
           297,   272,  2969,  3543, 28723, 17553,   349,  2651,   354,   871,
          2967, 14987, 13480,  2007, 28725,   390,  1162,   390,   871,  5024,
          7373,   297,  2309, 28725,  1955, 28725, 12502, 28725,   304,   798,
          2528, 28723, 17553,   659,   264,   680, 1

In [17]:
# Unload the model from GPU and RAM
del model          # Delete the model
gc.collect()       # Collect garbage
if device == 'cuda':
    torch.cuda.empty_cache()  # Clear CUDA cache